In [1]:
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy import stats
#import optshrink as opt # package we create
import numpy as np
# import scipy.io as sio
import h5py
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import seaborn as sns
from shapely.geometry import Point
import geopandas as gp
from geodatasets import get_path
from shapely.geometry import Polygon

In [2]:
clm_cols = ["lon", 'lat','var' ,'Area', 'year', 'sum', 'max', 'average'
         , 'min', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct'
         , 'Nov', 'Dec', 'Area_Name']
atm_cols = ['lon', 'lat', 'year', 'average']
elev_cols = ["lon", 'lat','var' ,'Area', 'elev', 'region']
soil_cols = ['lon','lat','var','Area','s1','s2','s3','tt','soil_text','region']
co2_cols = ['year', 'co2']

In [3]:

forest_grids = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/forest_gridcells.csv')


In [4]:
grid_monthly = ['/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/CanESM5xx1_ssp245_nirr.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/CanESM5xx1_ssp245_prec.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/CanESM5xx1_ssp245_tair.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/CanESM5xx1_ssp245_trange.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/CanESM5xx1_ssp245_vpr.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/CanESM5xx1_ssp245_wind.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/aot40_rcp45.csv'
]

grid_annual = ['/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/Ndep_rcp45.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/Nfer_crop.csv'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/Nfer_pas.csv'
]

grid = ['/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/cruigbpstxtlf.glb'
,'/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/crutbaselvlf.glb'
]

annual = '/group/moniergrp/TEM_Large_Ensemble/input_files/whole_time/co2_1700_2100.csv'

In [5]:
static = pd.read_csv(grid[0], names = soil_cols).merge(
     pd.read_csv(grid[1], names = elev_cols), on = ['lon','lat','Area','region'])
static


,lon,lat,var_x,Area,s1,s2,s3,tt,soil_text,region,var_y,elev
0,-180.0,65.0,TEXTURE,1040,30.68,52.38,17.45,0,IGBP,ASIA,ELEV,58.1
1,-180.0,65.5,TEXTURE,1030,30.39,52.86,17.28,0,IGBP,ASIA,ELEV,58.0
2,-180.0,66.0,TEXTURE,1245,53.73,29.95,16.59,0,IGBP,ASIA,ELEV,370.5
3,-180.0,66.5,TEXTURE,1220,49.77,33.84,16.70,0,IGBP,ASIA,ELEV,326.5
4,-180.0,67.0,TEXTURE,1195,51.21,32.43,16.66,0,IGBP,ASIA,ELEV,496.6
...,...,...,...,...,...,...,...,...,...,...,...,...
62603,179.5,67.5,TEXTURE,1170,54.51,29.19,16.56,0,IGBP,ASIA,ELEV,740.5
62604,179.5,68.0,TEXTURE,1145,49.34,34.26,16.72,0,IGBP,ASIA,ELEV,684.9
62605,179.5,68.5,TEXTURE,1120,49.43,34.28,16.59,0,IGBP,ASIA,ELEV,409.4
62606,179.5,70.5,TEXTURE,225,41.43,45.52,13.47,0,IGBP,ASIA,ELEV,136.2


In [6]:
co2 = pd.read_csv(annual,sep='\s+', names = co2_cols)
co2

,year,co2
0,1700,276.59
1,1701,276.62
2,1702,276.65
3,1703,276.67
4,1704,276.70
...,...,...
397,2097,601.80
398,2098,602.18
399,2099,602.50
400,2100,602.78


In [9]:
i = 0

In [7]:
for i in range(len(grid_annual)):

    file = grid_annual[i]
    base = file.rsplit('/', 1)[-1].rsplit('.', 1)[0]
    print(base)
    data = pd.read_csv(file, names = atm_cols)
    data=data.rename(columns = {'average':base})
    if i == 0:
        atm_chem = data
    else:
        atm_chem = atm_chem.merge(data, on = ['lon', 'lat', 'year'])

Ndep_rcp45
Nfer_crop
Nfer_pas


In [8]:
atm_chem.desribe()

,lon,lat,year,Ndep_rcp45,Nfer_crop,Nfer_pas
count,2.510581e+07,2.510581e+07,2.510581e+07,2.510581e+07,2.510581e+07,2.510581e+07
mean,1.986980e+01,3.058178e+01,1.900000e+03,2.296961e+02,6.969629e+02,2.387205e+02
std,8.644171e+01,3.165105e+01,1.157584e+02,2.880389e+02,2.525985e+03,2.459165e+03
min,-1.800000e+02,-5.600000e+01,1.700000e+03,1.627200e+00,0.000000e+00,0.000000e+00
25%,-6.450000e+01,7.500000e+00,1.800000e+03,6.415510e+01,0.000000e+00,0.000000e+00
50%,2.950000e+01,3.750000e+01,1.900000e+03,1.291837e+02,0.000000e+00,0.000000e+00
75%,9.450000e+01,5.700000e+01,2.000000e+03,2.739228e+02,0.000000e+00,0.000000e+00
max,1.795000e+02,8.350000e+01,2.100000e+03,8.847318e+03,3.608656e+04,9.824570e+04


In [15]:
pattern = r"time\/|CanESM5xx1_ssp245_"
i

file = grid_monthly[i]
parts = re.split(pattern, file)
parts
base = file.rsplit(pattern, 1)[-1].rsplit('.', 1)[0]
base = '.'.join(parts[-1].split('.')[:-1])
base

'aot40_rcp45'

In [16]:
import re

pattern = r"time\/|CanESM5xx1_ssp245_"

for i in range(len(grid_monthly)):
# for i in range(3,7):

    file = grid_monthly[i]
    parts = re.split(pattern, file)
    base = file.rsplit(pattern, 1)[-1].rsplit('.', 1)[0]
    base = '.'.join(parts[-1].split('.')[:-1])
    print(base)
    data = pd.read_csv(file, names = clm_cols)[['lon', 'lat', 'year','average','min','max']]
    data=data.rename(columns = {'average':base+'_avg'
                               ,'min':base+'_min' 
                                ,'max':base+'_max' 
                               })
    # print(data.dtypes)
    if i == 0:
        climate = data
    else:
        climate = climate.merge(data, on = ['lon', 'lat', 'year'])

nirr
prec
tair
trange
vpr
wind
aot40_rcp45


In [17]:
input_data = forest_grids.merge(
    climate, on = ['lon','lat']).merge(
    atm_chem, on = ['lon','lat','year']).merge(
static, on = ['lon','lat']
).merge(co2, on = 'year')

In [18]:
print(len(input_data))
print(len(climate))
input_data.to_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/whole_time_CanESM5xx1_ssp245_global_input_vars.csv' ,index=False)

12271803
25105808


In [12]:
test_region.columns

Index(['lon', 'lat', 'year', 'variable', 'stand_age_interval_min',
       'forest_type', 'current_veg', 'community_type', 'silt_clay', 'region',
       'file', 'land_area', 'cohort_area', 'monthly_mean_weight', 'Jan_weight',
       'Feb_weight', 'Mar_weight', 'Apr_weight', 'May_weight', 'Jun_weight',
       'Jul_weight', 'Aug_weight', 'Sep_weight', 'Oct_weight', 'Nov_weight',
       'Dec_weight', 'monthly_mean', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
       'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec', 'PFT_group'],
      dtype='object')

In [13]:
model_data = test_region.loc[(test_region['variable']==' VEGC ') & (test_region['PFT_group'] == 'Temperate Coniferous')
               ,['lon', 'lat', 'year','stand_age_interval_min','silt_clay','monthly_mean']].merge(
 input_data, on = ['lon','lat','year']
)

In [14]:
model_data.columns

Index(['lon', 'lat', 'year', 'stand_age_interval_min', 'silt_clay',
       'monthly_mean', 'tswrf_v11', 'tmp', 'precip', 'dtr', 'vpr', 'wind',
       'Ndep_Trendy', 'Nfer_crop', 'Nfer_pas', 'var', 'Area', 'elev', 'region',
       'co2'],
      dtype='object')

In [16]:
model_data.to_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/region_3_test_model_data_conif_vegc.csv', index=False)

In [3]:
model_data = pd.read_csv('/home/smmrrr/TEM_Analysis/TEM_Analysis/region_3_test_model_data_conif_vegc.csv')

In [10]:
unique_bins = model_data.stand_age_interval_min.unique()
bin_to_ordinal = {bin_val: idx for idx, bin_val in enumerate(unique_bins)}
print(bin_to_ordinal)
model_data['ordinal_stand_age'] = model_data['stand_age_interval_min'].map(bin_to_ordinal)
model_data['ordinal_stand_age'].describe()

{150: 0, 0: 1, 5: 2, 10: 3, 15: 4, 20: 5, 25: 6, 30: 7, 35: 8, 40: 9, 45: 10, 50: 11, 55: 12, 60: 13, 65: 14, 70: 15, 75: 16, 80: 17, 85: 18, 90: 19, 95: 20, 100: 21, 125: 22}


count    287630.000000
mean          5.893586
std           5.567873
min           0.000000
25%           1.000000
50%           5.000000
75%           9.000000
max          22.000000
Name: ordinal_stand_age, dtype: float64

In [ ]:
# X = np.array(model_data.loc[model_data['year']>=2021,['tswrf_v11', 'tmp', 'precip', 'dtr', 'vpr', 'wind','stand_age_interval_min']])
X = np.array(model_data.loc[model_data['year']>=2000,['ordinal_stand_age', 'tmp']])
y = np.array(model_data.loc[model_data['year']>=2000,['monthly_mean']])

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel

kernel = DotProduct() + WhiteKernel()

gpr = GaussianProcessRegressor(kernel=None,
      random_state=0).fit(X, y)

gpr.score(X, y)